## Filter by coordinates

Filtro i dati in formato json ottenuti nella prima fase, in base alle loro coordinate: se si trovano all'interno di un poligono predefinito (l'area che ci interessa), allora li filtro, li pulisco e prendo solo le informazioni a noi utili (nome utente, coordinate, ecc.) e li inserisco in un dataframe, che chiamo "df".

In [1]:
import numpy as np
import json
import os
import gzip as gz
from shapely.geometry import *
import pandas as pd
from pandas import DataFrame 

from createDataframe import CreateDF

In [2]:
# MAIN

#le coordinate sono: [(43.7359, 10.4269), (43.6955, 10.3686)]
#costruisco un rettangolo con queste coordinate
coords = box(43.7359,
           10.4269,
           43.6955,
           10.3686)
print('Rectangle coordinates', coords2)

directory_base = '../'

for folder in os.listdir(directory_base):
    folder_name = os.path.basename(folder)
    if folder_name.startswith('uidFiles'):
        #creo il dataframe con le colonne
        df = pd.DataFrame(columns=['Screen_name','UserID', 'TweetID', 'Coords', 'Lat', 'Lon', 'Created_At', 'Text'])
        #salvo i file su entries dalla cartella "uidFiles"
        entries = os.listdir('../'+folder_name)
        print('In '+folder_name+' ci sono '+str(len(entries))+' files') #stampo quanti file ci sono
        #creo la lista per appendere i file con i tweets che hanno le coordinate (perché molte sono null)
        emptyFileWithCoord = []
        
        createDataframe = CreateDF(entries, df, folder_name, emptyFileWithCoord, coords)
        
        fileWithCoord = createDataframe.createFilesWithCoord(entries, folder_name, emptyFileWithCoord)
        df = createDataframe.createDf(entries, df, folder_name, fileWithCoord, coords2) #creo il df
        
        #saving the dataframe
        df_path = '../csv/df_'+folder_name+'.csv'
        df.to_csv(df_path)
        print('Created'+ df_path)

Rectangle coordinates POLYGON ((43.6955 10.4269, 43.6955 10.3686, 43.7359 10.3686, 43.7359 10.4269, 43.6955 10.4269))
In uidFiles_2016 ci sono 2521 files
Created../csv/df_uidFiles_2016.csv


In [3]:
df

,Screen_name,UserID,TweetID,Coords,Lat,Lon,Created_At,Text
x,madikeeper12,868809325,779072240994234368,"[43.72666207, 10.41268069]",43.726662,10.412681,Thu Sep 22 21:37:51 +0000 2016,Cieli infuocati.\n\n#picoftheday #quotesofthed...
x,madikeeper12,868809325,781615843406819329,"[43.72666207, 10.41268069]",43.726662,10.412681,Thu Sep 29 22:05:13 +0000 2016,Prospettive.. \nunite a casa #ilselfone\n#team...
x,madikeeper12,868809325,781870800156499968,"[43.72666207, 10.41268069]",43.726662,10.412681,Fri Sep 30 14:58:19 +0000 2016,"Non occorre essere matti per lavorare qui, ma ..."
x,madikeeper12,868809325,780003801260404736,"[43.7167, 10.3833]",43.716700,10.383300,Sun Sep 25 11:19:32 +0000 2016,RunOnSunDay 🏃🏽‍♀️☀️\n#run #running #runner #ni...
x,madikeeper12,868809325,779443101123260417,"[43.70561, 10.42059]",43.705610,10.420590,Fri Sep 23 22:11:31 +0000 2016,La vita è come la fotografia sono necessari i ...
...,...,...,...,...,...,...,...,...
x,antoniocassisa,358042635,781879291911016448,"[43.7167, 10.3833]",43.716700,10.383300,Fri Sep 30 15:32:04 +0000 2016,"I mì ómini \n#son #figli #boys @ Pisa, Italy h..."
x,SefaMermer,293157588,780753755830677504,"[43.7167, 10.3833]",43.716700,10.383300,Tue Sep 27 12:59:35 +0000 2016,#love #tbt #tagforlikes #TFLers #tweegram #pho...
x,SefaMermer,293157588,780756143668953088,"[43.7167, 10.3833]",43.716700,10.383300,Tue Sep 27 13:09:05 +0000 2016,#love #tbt #tagforlikes #TFLers #tweegram #pho...
x,matteluca89,494389053,779638196258811904,"[43.71544235, 10.40051616]",43.715442,10.400516,Sat Sep 24 11:06:45 +0000 2016,Last saturday I went out with my #chinese teac...


### Costruisco la mappa

In [5]:
import folium
from folium import plugins
from shapely.geometry import Point, Polygon, LineString

In [6]:
#le coordinate sono: [(43.7359, 10.4269), (43.6955, 10.3686)]
lat = [43.7359, 43.6955]
lon = [10.4269, 10.3686]

lat_mean = np.mean(lat)
lon_mean = np.mean(lon)

In [7]:
coords = df['Coords']
m = folium.Map(location=[lat_mean, lon_mean], tiles='Stamen Toner', zoom_start=13.2, control_scale=True)
 
for x in coords:
    folium.Marker(x).add_to(m)
 
m